# Course: Web Mining Project (5944P)

Master of Computer Science@University of Passau

by 

[__Michael Granitzer__ (michael.granitzer@uni-passau.de)]( http://www.mendeley.com/profiles/michael-granitzer/)





__License__

This work is licensed under a [Creative Commons Attribution 3.0 Unported License](http://creativecommons.org/licenses/by/3.0/)


In [2]:
#some configuration stuff to be ignored
#loads section numbering https://github.com/ipython/ipython/wiki/Extensions-Index
%reload_ext secnum
%secnum

# Motivation - the Why?

The Web has become the largest source of information in the world

<img src="files/images/web-data-infographic.jpeg"/>
Source: http://royalonlinemedia.wordpress.com/2013/02/28/new-journalism-data-infographics/

**Some Questions**
- But what can we learn from this data?
- What kind of services can we build around it?
- What insights and knowledge is hidden in the pattern?
- Does the data tell us something on our own behaviour?
- Does it allow to answer sociological/psychological questions?
- ....

## What is Web Mining?

<div class ="alert alert-sucess">
**Web mining** is the use of data mining techniques  to automatically discover and extract information from Web documents/services (Etzioni, 1996)
</div>

- **Web Content Mining:** Extract knowledge out of documents and user content in the web (e.g. Filtering messages in social networks)
- **Web Structure Mining:** Analyse the graph structure of the web and identify interesting patterns (e.g. Page Rank)
- **Web Usage Mining:** Analyse the usage of web systems and extract interesting patterns (e.g. Analyse Navigation Paths)

## 1.2. Google Flu Trends - an Example

[Google Flu Trends](http://www.google.org/flutrends/de/#DE) - Can we predict flu outbreaks from search behaviour?

<img src="files/images/flu-trends.png"\>

## Fields of Application

- Web Search 
- Personalisation in E-Commerce (e.g. Amazon)
- Recommender Systems
- Advertisement/Tracking User Behaviour
- Digital Libraries
- Web Site Optimization
- Social Media Analytics (e.g. for Marketing)
- Sociology Research

# A small Use Case Example - Personal Filtering of Twitter Messages

[Twitter](http://Twitter.com) - a social network for short messages
<img src="files/images/twitter example.png"/>

Let's consider a small use case: lets say we need a program that filters Twitter Messages (Tweets) according to our interest. The program should conduct the following functions:

1. Tweets should be crawled from Twitter
2. A user somehow must express what is relevant and what is not relevant for them
3. Our program needs to represent this notion of relevance somehow
4. Our program should store the relevant tweets in the users inbox/pc
5. Our program should allow the user to give feedback on what tweet is relevant/not relevant and adapt its strategy
5. Our program should group relevant tweets together according to their topic
6. Our program should show us the development of a topic over time



<div class="aler alert-info">
How would you solve this problem?
</div>

## Sketching the Solution


### Step 1 - Data Gathering

Crawling/Gathering Data from Twitter. In case of Twitter it is easy, since they provide an [API](https://dev.twitter.com/).

Crawling abritrary web pages requires to write more complicated crawlers and consider different formats. 



### Step 2 - Machine Learning - Classification 

By using Machine Learning methods, we can automatically classify tweets belonging to a certain class. So called **Supervised Learning Methods** learn rules (e.g. mathematical models)from those examples to classify future, yet unkown examples.

So a user would mark tweets as relevant/non-relevant. The tweets are then used to estimate a model to predict future tweets.



### Step 3 - Machine Learning - Clustering 

Clustering belongs to the **Unsupverised Machine Learning** methods. They group similar items toghether while maintaining maximal dissimilarity between groups.

Using clustering allows us to group tweets which are similar together, while the groups itself are most dissimlar. So tweets from two different groups do not have a lot in common. This seems to be a reasonable approximation for our topics.


### Step 4 - Visualising Data

After obtaining topics in form of groups of similar tweets, we need to display the number of tweets per topic over the time. Could look like:

<img src="files/images/10SteamGraph2.jpg"/>

Source: [Twitter StreamGraph](http://www.neoformix.com/Projects/TwitterStreamGraphs/view.php)

## Digging Deeper - How to do the heavy lifting?

**Machine Learning** is the core of our application. So how could this work? 


### A Linear Model for Classifying Tweets

Lets assume relevance of a tweet only depends on the word occuring in a tweet. So a model for classifying tweets could be simply to 

1. Assign every word a with how relevant it is
2. Sum up all the relevance for every word in a tweet
3. If the sum exceeds a certain threshold, i.e. there are a lot of relevant words in a tweet, the whole tweet must be relevant


####  More formally

Let

- $T$ be a list of tweets with denoting $t_j$ als j-th tweet
- $D$ be the dictionary of all words in all tweets 
- $d_i\in D$ be the i-th word in the dictionary
- $w_i\in W$ be the relevance of $d_i$. Let $w_i>0$ denotes positive relevance and let $w_i<0$ denotes negative relevance
- $f_{i,j}$ denotes how often $d_i$ occures in $t_j$




Then a tweet relevance score can be calculated as weighted sum of word occurences:

$$
relevance(t_j) = \sum_{d_i\in D} w_i*f_{i,j}
$$


And we can classify a tweet as relevant when its relevance score exceeds a certain threshold:

$$
classify(t_j) = \left\{
  \begin{array}{lr}
    0 & : relevance(t_j)-\theta<0 \\
    1 & : relevance(t_j)-\theta>0 \\
  \end{array}
\right.
$$




#### Representing Tweets as Matrix

For a more compact representation, we can represent each tweet as vector $\overline{f_j}$ of word occurences and all tweets as matrix $F$. Hence, classifying becomes vector/matrix operations:

$$
F*\overline{w}-\theta>0
$$

with $\overline{w}$ being the vector of relevance weights (note that strictly mathematically speaking $\theta$ must be a vector. Most machine learning methods utilize such a matrix representation. Obtaining this matrix representation is called "preprocessing".



<div class="alert alert-success">
**As a consequence, for successfully applying web mining techniques we need to**<br>
<ol>
    <li> convert our data to matrices and vectors (**preprocess data**)
    <li> apply matrix operations/linear algebra (**numerical computations**)
</ol>
</div>

#### How to obtain the weights?

Trying to set the weights manually can be a tedious at its best. For practical scenarios it can be considered to be impossible. So we would like to have some machinery to estimate the weights.

In order to get such a machinery working, we need to have

1. manually labelled data, i.e. Tweets, that are considered to be relevant and some non-relevants
2. an algorithm to "learn" the weights based on the manually labelled data



<div class = "alert alert-info">
How would you adjust the weights?
</div>

*Observation:

1. if $relevance(t_j)-\theta>0$ &nbsp; but&nbsp; $t_j$&nbsp; is **not relevant** to the user, that means the relevance $w_i$ for all words $d_i$ in $t_j$ was to high and need to be reduced. 
2. if $relevance(t_j)-\theta<0$&nbsp; but&nbsp; $t_j$&nbsp; is **relevant** to the user, that means the relevance $w_i$ for all words $d_i$ in $t_j$ was to low and need to be increased. 






So lets define $y_j$ as being 1 for a relevant tweet and 0 otherwise. Then we can define the error as 
$$
error_{j} = classify(t_j)- y_j
$$ 

and devise a simple update rule for tweet $t_j$ as:

$$
w_i = w_i - \eta*error_{j}*f_{i,j}
$$

with $\eta\in[0,1]$ being a small constant called learning rate. 

So the update of a weight for a tweet looks like

|$y_j$|$classify(t_j)$|new $w_i$|
|----|----------------|---------|
| 1  | 0              |increase |
| 0  | 1              |decrease |
| 0  | 0              |no update|
| 1  | 1              |no update|

Doing this for all tweets $t_j\in T$ updates the weights correspondingly. You might need to do that during several iterations.

There are numerous methods for solving this more efficently and more accurately. We will review some of those methods during this course.


<div class="alert alert-success"> 
<ol>
 <li> The algorithm sketched here is called stochastic gradient descent and the model is a linear model. More details later on.
 <li> Every supervised learning algorithm needs feedback to adjust it's model accordingly. In nearly all cases these are single examples labeled with the target value. 
 <li> Manually labelled data is in most settings the most scarce resource
</ol>
</div>

### Extracting topics - Clustering Tweets

We can distinguish between relevant and non-relevant tweets. But how to group the relevant tweets into single topics?

First we need to define what a topic is:

We define a topic as a group of tweets **similar in content**.



<div class="alert alert-info">
So how would you determine the similarity between two tweets?
</div>

#### Calculating the similartiy between tweets

*Observation:* Two tweets which are similar share similar words

*Hypothesis:* The more words are shared, the higher the similarity between two tweets


**Formally**

Let $\overline{f_j}$ be the word frequency vector for a tweet, then the similarity between two tweets $t_j$ and $t_i$ can be calculated as 

$$
 sim_{j,i}= \overline{f_j}^T\cdot\overline{f_i}
$$


In order to get a measure in the intervall $[0,1]$ we need to scale both vectors to unit length:

$$
 sim_{j,i}= \frac{\overline{f_j}^T\cdot\overline{f_i}}{|\overline{f_j}||\overline{f_i}|}
$$



<div class ="alert alert-success">
This measure is also known as **cosine measure** between two vectors. It measures the cosine of the angle between two measuers and is widley used in web mining (more accurately for sparse vector spaces)
</div>

#### Clustering - Grouping similar tweets together

We can now estimate the similarity of two tweets. But how to get groups of similar tweets and can a tweet be in more than one group?

In order to ease the task we make the followig assumption: *One tweet belongs to one group/topic*

<div class="alert alert-info"> 
Given the similarity measure, how could you find groups of similar tweets?
</div>

**Idea**

1. Lets choose $k$ tweets which represent different topics as starting point. 
2. now calculate the similartiy between all tweets $t_j\in T$ and all $k$ representative tweets
3. Assign tweet $t_j$ to the topic with the highest similarity

<div class="alert alert-success">
This is the first step in the well known **k-means Clustering Algorithm**. There are many more clustering algorithm and we will look into some of them (and their properties)
</div>

<div class="alert alert-info">
Do you know why Steps 1.-3. are not sufficient for finding a good grouping?
</div>

# What I offer to you

<div class="alert alert-success">
We will not be able to cover all aspects of web mining. However, the **goal is to help you on your first steps to mine the web**. 

<p> Mining means to extract new, potentially interesting and valid patterns from web sources.
</div>

## Lecture Overview

There are a lot of tools and technologies out there to do so. In this course, we will use the Python programming language and its tons of libraries. 

To give you the necessary abilities to mine the Web with Python, **I** will give lectures and practical exercises on the following topics:

<div class="alert alert-warning">
<ol>
<li> A short introduction to Data Science with Python
<li> An introduction to the Python Programming Language
<li> Important libraries for Data Science (plus the minimum necessary underlying theory)
    <ol>
       <li>**Crawling Twitter** and **Cleaning HTML**
       <li>**Preprocessing Text:** NLTK - Natural Language Toolkit 
       <li>**Numerical Computation:** Numpy - Efficient Numerical Computations       
       <li>**Machine Learning:** Scikit Learn - A easy-to-use machine learning library 
       <li> **Visualisation:** Matplotlib
    </ol>
</ol>
</div>

<p>
We will focus mostly on practical aspects. Theory (e.g. algorithmic details) is only covered where necessary. 

However, I recommend that you also start obtaining the theoretical background, especially in machine learning. Only with a deep theoretical understanding you will be able to truely master the subject.

## Your Projects

Experience is the best teacher. So in order to master the subject, you need to conduct a web mining project on predefined topics.

### Project Topics

Our main data source will be [Twitter](http://www.twitter.com), simply because it provides an easy to use data source. 

As a second data source, web pages in form of the [WebKB data set](http://www.cs.cmu.edu/~webkb/) will also be available. 

The following topics will be available

<div class="alert alert-warning">
<ul>
<li> Automatic Shitstorm Detection in Twitter (Group of 2-3)
<li> Reveal topic, event and person distributions in Twitter (Group of 2-3)
<li> Sentiment analysis on Twitter Data (Group of 2)
<li> Web page classification (Group of 1-2)
</ul>
</div>


Every topic is associated with a set of questions, that should be answered by you. 

# How - Timeline and Grading   

## Timeline


- **October to December**: Building up the basics. Lectures by me supplemented with small exercises for you
- **December 5th**: Group and topic assignment
- **December to January**: Time to work on your project
- **End of January**: Report your findings to your colleagues in a brief presentation (10 minutes)

All details are provided in [Stud.IP](https://studip.uni-passau.de/studip/seminar_main.php?auswahl=0733c229176a53b95f2f54c856755da8)

## Grading



Grading is based solely on your project:
1. How accurate can you solve the task at hand?
2. How good do your results generalize?
3. What are the most important properties of the data to solve your task at hand?
4. What is the most suitable algorithm and why?
5. How correct is your justification of all the above points?

Since you can work in groups, you  must make clear how fullfilled which role. The following roles can be foreseen:

- **Data Gathering and Preprocessing:** Fetch the data and bring it in a usable form
- **Data Analysis Process:** Setup the data analysis process by selecting algorithms and methods
- **Evaluation:** Setup the evaluation procedure.

The single roles are interwined and depend on each other. 

All your results must be summarized in an IPython Notebook. I will have a talk with every group regarding the contents of the notebook (Abgabegespräch) in order to determine quality and correctness of your results and to make sure the work has been done by you as claimed.


The course counts as "Wahlfach Alg+Math" and as "Wahlpflichtpfach InfKomm" for the Master Informatik (StPO 2012)

## Rules


- **There is no mandatory attendance**: If you already know a particular topic or if you want to learn it on you own, you dont have to participate the weekly lecture. Note: You are learning for you, not for me.

- **Teamwork yes, plagiarism no**: Team work and discussing problems is important. However, copying code is forebidden and counts as plagiarism. If i encounter plagiarism in a group, all members will be graded with 5. 

- **You must be able to explain your work to me**: If one member of a group can not explain to me what he/she did *in detail*, i consider the work has been done by somebody else and will grade it as 5.

- **Asking questions is not forbidden**: Please, have the courage to ask questions if you are not understanding something. 

<p>
<div class="alert alert-error">
**The most important rule:** Have fun digging into data
</div>



# Reading Materials

** All material presented in the lecture is available on GitHub**
<div class="alert alert-success">

[LODA - Lecture Notes on Data Analysis](https://github.com/mgrani/LODA-lecture-notes-on-data-analysis)
</div>

**Recommended Readings**

<div class="alert alert-success">

Machine Learning, T. Mitchell, McGraw Hill 1997 (http://www.cs.cmu.edu/~tom/mlbook.html)
</div>
<div class="alert alert-success">
Pang-Ning Tan, Michael Steinbach, Vipin Kumar, Introduction to Data Mining, 2006, Pearson Education
</div>

** Recommended Readings for a Short Introduction to Web Mining **

- Raymond Kosala and Hendrik Blockeel. 2000. Web mining research: a survey. SIGKDD Explor. Newsl. 2, 1 (June 2000), 1-15. DOI=10.1145/360402.360406 http://doi.acm.org/10.1145/360402.360406


Further, individual material will be presented within every lecture.

You can also attend the Course "Machine Learning and Data Mining" or "Text Mining" for obtaining a deeper theoretical foundation.